In [5]:
!pip install openpyxl

In [77]:
import tensorflow
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import shutil

In [7]:
df = pd.read_excel('predictions2.xlsx')

In [12]:
df = df.drop('Unnamed: 0',axis=1)

In [26]:
df['filename'][0].split('/',4)

['', 'workspace', 'train', 'dataset', 'test/Type_1/35 (2).jpg']

In [27]:
test_data_path = "D:/Project_CVX_Classification/dataset/"

In [35]:
df["Local_Data_Path"] = df['filename'].apply(lambda x:test_data_path + x.split('/',4)[-1])

In [40]:
test_array = []
test_labels = []
IMG_DIMS = (224, 224)

for i in tqdm(range(len(df))):
    img_path = df["Local_Data_Path"][i]
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=IMG_DIMS,
                     interpolation=cv2.INTER_CUBIC)
    img = np.array(img, dtype=np.float32)
    test_array.append(img)
    test_labels.append(df['label'][i])


100%|██████████| 600/600 [00:03<00:00, 174.20it/s]


In [41]:
test_labels= np.array(test_labels)
test_imgs_scaled = np.array(test_array) / 255.
test_imgs_scaled.shape, test_labels.shape

((600, 224, 224, 3), (600,))

In [42]:
model = tensorflow.keras.models.load_model("D:/CVX/Cervix_Cancer_Classification/densenet_cvx.h5")

In [43]:
predictions = model.predict(test_imgs_scaled) 
predicted_labels = np.argmax(predictions, axis=1)

19/19 [==============================] - 43s 2s/step


In [44]:
df['Local_Pred'] = predicted_labels

In [49]:
df2 = df[['label','Predicted_Labels','Local_Pred']]

In [53]:
pd.Series(df2['Predicted_Labels'] == df2['Local_Pred']).value_counts()

True    600
Name: count, dtype: int64

In [56]:
df3 = df[["label","True_Labels","Predicted_Labels"]]

In [67]:
pd.Series(df3[df3['label'] == "Type_1"]['Predicted_Labels'] == 0).value_counts()

Predicted_Labels
True     115
False     85
Name: count, dtype: int64

In [71]:
115/130

0.8846153846153846

In [72]:
106/130

0.8153846153846154

In [75]:
123/130

0.9461538461538461

In [76]:
344/390

0.882051282051282